In [1]:
from context import *

# Pauli Algebra (`paulialg`)

## Basic Usage

### Operator Construction

#### Pauli Operator

A **Pauli operator** can be constructed using the `pauli()` constructor.

In [2]:
qst.pauli('XXIYZ')

 +XXIYZ

**Specify the Phase Factor**

By default the operator has a $+1$ phase factor in the front. To specify other phase factors($\pm1$ or $\pm \mathrm{i}$), use `'+'`, `'-'`, `'i'` indicators before the Pauli string.

In [3]:
qst.pauli('-X'), qst.pauli('iX'), qst.pauli('-iX')

( -X, +iX, -iX)

It is also possible to assign the phase factor by scalar mutiplication.

In [4]:
-qst.pauli('X'), 1j*qst.pauli('X')

( -X, +iX)

**Specify the Pauli String**

Other methods to specify a Pauli string:
* construct from a tuple / list / array of indices (`0` = `I`, `1` = `X`, `2` = `Y`, `3` = `Z`)

In [5]:
qst.pauli((0,1,2,3)), qst.pauli([0,1,2,3]), qst.pauli(numpy.array([0,1,2,3]))

( +IXYZ,  +IXYZ,  +IXYZ)

* construct from a dictionary that maps positions to indices. (*Note*: using this method must also provide the total number of qubits as the second argument, because the qubit number can not be infered from the dictionary alone.)

In [6]:
qst.pauli({1:'X', 4:'Y', 5:'Z'}, 6), qst.pauli({1:1, 4:2, 5:3}, 6),

( +IXIIYZ,  +IXIIYZ)

#### Pauli Operator List

A **list of Pauli operators** can be constructed by the `paulis()` constructor.

In [7]:
qst.paulis('X', '-Y', 'Z')

 +X
 -Y
 +Z

It can take a generator and iterate through its elements to construct a list of Pauli operators.

In [9]:
qst.paulis([{i:'Z'} for i in range(4)], N = 4)

 +ZIII
 +IZII
 +IIZI
 +IIIZ

It can also take a iterable (tuple / list / set) and convert it to a list of Pauli operators.

In [10]:
objs = ['XX', 'YY', 'ZZ']
qst.paulis(objs)

 +XX
 +YY
 +ZZ

#### Size Information

For Pauli operator, `.N` returns the number of qubits (size of system) that the operator acts on.

In [11]:
qst.pauli('IIII').N

4

For Pauli operator list, `.L` returns the number of operators in the list and `.N` returns of the number fo qubits in the system.

In [12]:
plst = qst.paulis('II','XX','YY','ZZ')
plst.L, plst.N

(4, 2)

In [13]:
len(plst)

4

#### Selection and Slicing

Select a single element in the Pauli operator list.

In [14]:
plst[1]

 +XX

Select a range of operators in the Pauli operator list.

In [15]:
plst[0:3]

 +II
 +XX
 +YY

In [16]:
plst[-2:]

 +YY
 +ZZ

In [17]:
plst[::2]

 +II
 +YY

It is also allow to be selected by a index array or a boolean mask.

In [18]:
plst[numpy.array([2,1,1,0,3])]

 +YY
 +XX
 +XX
 +II
 +ZZ

In [19]:
plst[numpy.array([True,False,False,True])]

 +II
 +ZZ

### Operator Algebra

#### Scalar Product

Pauli operator and operator list can be multiplied with a scalar.
* If the scalar is a phase factor (as power of imaginary unit), the phase will be multiplied to the operator.

In [20]:
-qst.pauli('X'), 1j*qst.pauli('X')

( -X, +iX)

   For operator list, the scalar multiplication broadcast to every operator in the list.

In [21]:
-qst.paulis('II','XX','YY','ZZ')

 -II
 -XX
 -YY
 -ZZ

* if the scalar is beyond a phase factor, the Pauli operator will be promoted to a **Poly monomial** (i.e. Pauli operator equipted with a complex coefficient.)

In [22]:
2*qst.pauli('X'), -1.5*qst.pauli('X'), (-0.3+0.4j)*qst.pauli('X')

(2 X, -1.50 X, (-0.30+0.40j) X)

However, Pauli opreator list does not support scalar multiplication beyond the four phase factors. (Because there is no canonical meanding for a list of Pauli monomials).

#### Linear Combination

Pauli operators can be linearly combined in to a **Pauli polynomial**.

In [23]:
qst.pauli('XX') + qst.pauli('YY') - 0.5 * qst.pauli('ZZ')

-0.50 ZZ +1 XX +1 YY

Adding Pauli operators with any number, the number will be promoted to the number times identity operator automatically. For example, a projection operator can be written as

In [24]:
(qst.pauli('ZZ') + 1)/2

0.50 II +0.50 ZZ

Operators can be summed up with python built-in function `sum()`.

In [25]:
sum(qst.paulis('II','XX','YY','ZZ'))

1 II +1 ZZ +1 XX +1 YY

#### Dot Productor (Matrix Multiplication)

Dot productor (composition) of Pauli operators is implemented as the matrix multiplication `matmul`, which can be implemented using the operand `@`.

In [26]:
qst.pauli('X') @ qst.pauli('Y'), qst.pauli('Y') @ qst.pauli('X')

(+iZ, -iZ)

The dot product of two Pauli operators is still a Pauli operator. However if any one of them is Pauli monomial, the result will also be Pauli monomial.

In [27]:
(3*qst.pauli('X')) @ qst.pauli('X'), (2*qst.pauli('X')) @ (3*qst.pauli('Z'))

(3 I, (0.00-6.00j) Y)

Dot product of Pauli polynomials will be expanded.

In [28]:
poly = qst.pauli('XX') + qst.pauli('YY') - 0.5 * qst.pauli('ZZ')
poly @ poly

0.25 II +0.50 YY +0.50 XX +0.50 YY +1 II -1 ZZ +0.50 XX -1 ZZ +1 II

Terms will not be combined automatically. To combine them, the `.reduce()` method should be explicitly called.

In [29]:
(poly @ poly).reduce()

2.25 II -2 ZZ +1 XX +1 YY

This allows the user to get control of the reduction, to avoid unnecessary reductions in the intermediate calculation.

#### Identity and Zero

Identity and zero operators can be constructed by `pauli_identity(N)` and `pauli_zero(N)` given the qubit number `N`.

In [30]:
qst.pauli_identity(5), qst.pauli_zero(5)

(1 IIIII, 0 IIIII)

This makes it convinient to refer to these operators in the calculation.

In [32]:
qst.pauli_zero(5) + qst.pauli('XXXXX'), qst.pauli_identity(5) @ qst.pauli('XXXXX')

(1 XXXXX, 1 XXXXX)

### Properties and Type Conversion

There are four different types of objects involved in the above discussion.
* `Pauli` (base class): a Pauli operator (in the Pauli group).
    * `PauliMonomial` (subclass of `Pauli`): a Pauli operator with general coefficient (outside the Pauli group). 
* `PauliList` (base class): a list of Pauli operators.
    * `PauliPolynomial` (subclass of `PauliList`): a linear combination of Pauli operators (stored as Pauli operator list together with combination coefficients.)

In [33]:
type(qst.pauli('X')), type(2*qst.pauli('X')), type(qst.paulis('X','X')), type(sum(qst.paulis('X','X')))

(qst.paulialg.Pauli,
 qst.paulialg.PauliMonomial,
 qst.paulialg.PauliList,
 qst.paulialg.PauliPolynomial)

#### Properties

As subclasses, `PauliMonomial` and `PauliPolynomial` inherit the related size properties from their parent classes.

In [34]:
(5*qst.pauli('XYZ')).N

3

In [35]:
poly = sum(sum(qst.paulis('II','XX','YY','ZZ')))
poly, poly.L, poly.N

(1 II +1 ZZ +1 XX +1 YY, 4, 2)

`PauliPolynomial` can be selected and sliced as a list.

In [36]:
poly[1], poly[:2], poly[::2]

(1 ZZ, 1 II +1 ZZ, 1 II +1 XX)

In [37]:
poly[numpy.array([1,1,1,2])].reduce()

3 ZZ +1 XX

In [38]:
poly[numpy.array([True,False,False,True])]

1 II +1 YY

#### Type Conversion

`Pauli` can be converted to `PauliMonomial`.

In [39]:
qst.pauli('XX').as_monomial()

1 XX

`Pauli`, `PauliList`, `PauliMonomial` can all be converted to `PauliPolynomial`.

In [40]:
qst.pauli('XX').as_polynomial()

1 XX

In [41]:
qst.paulis('II','XX','YY','ZZ').as_polynomial()

1 II +1 XX +1 YY +1 ZZ

Automatic type conversion enables the algebra to be carried out among different classes with great flexibiliity.
* When `Pauli` is multiplied (`*`) by a generic number (beyond powers of the imaginary unit), it is converted to `PauliMonomial`.
* When `Pauli` or `PauliMonomial` is added (`+`) or subtracted (`-`) with other Pauli objects, they are converted to `PauliPolynomial`.
* The dot product (`@`) generally returns `PauliPolynomial`, unless the two Pauli objects are both `Pauli`, in which case it returns `Pauli`.

### Clifford Transformation

`PauliList` provides useful methods to implement Clifford transformations efficiently on all Pauli operators together. The same methods are available to all its subclasses (including `PauliPolynomial`, `CliffordMap`, `StabilizerState`).

#### Clifford Rotation

A Clifford rotation is a $\mathbb{Z}_4$ rotation in the Clifford group generated by a single Pauli operator, which takes the form of
$$U=e^{\frac{i\pi}{4}\sigma}=\frac{1}{\sqrt{2}}(1+i \sigma).$$
Every Pauli operator is transformed by $\sigma \to U^\dagger \sigma U$. The Clifford rotation can be applied by the method `.rotate_by(gen)` (given the generator `gen`). The operation is in-place (meaning that the operators in the Pauli list will be modified).

In [43]:
qst.paulis('II','XX','YY','ZZ').rotate_by(qst.pauli('XI'))

 +II
 +XX
 +ZY
 -YZ

#### Clifford Map

A Clifford map is a generic clifford transformation by specifying how each single Pauli operator gets mapped to. It can be listed as a table

In [44]:
cmap = qst.random_clifford_map(2)
cmap

CliffordMap(
  X0-> +YX
  Z0-> -ZI
  X1-> +IX
  Z1-> -ZZ)

It can be applied by the method `.transform_by(cmap)` (given the Clifford map `cmap`). 

In [45]:
qst.paulis('II','XX','YY','ZZ').transform_by(cmap)

 +II
 +YI
 -YZ
 +IZ

#### Masked Transformation

Clifford transformation can be applied to a subsystem of qubits specified by a mask.

In [48]:
mask = numpy.array([True,False,False,True])
qst.paulis('IIII','XXXX','YYYY','ZZZZ').rotate_by(qst.pauli('XY'), mask)

 +IIII
 -IXXZ
 +ZYYI
 +ZZZZ

In [49]:
mask = numpy.array([True,False,False,True])
qst.paulis('IIII','XXXX','YYYY','ZZZZ').transform_by(cmap, mask)

 +IIII
 +YXXI
 -YYYZ
 +IZZZ

## Algorithm Details

### Internal Representation

#### Binary Representation of Pauli Operators

Any Pauli operator can be specified by two one-hot (binary) vectors $x$ and $z$ ($x_i,z_i=0,1$ for $i=1,\cdots,N$):
$$\sigma_{(x,z)}=\mathrm{i}^{x\cdot z}\prod_{i=1}^{N}X_i^{x_i}\prod_{i=1}^{N}Z_i^{z_i}.$$
* The binary vector $x$ (or $z$) specifies the qubits where the $X$ (or $Z$) operator acts ($Y$ operator acts at where $X$ and $Z$ act simultaneously).
* **Multiplication** of two Pauli operators
$$\sigma_{(x,z)}\sigma_{(x',z')}=\mathrm{i}^{p(x,z;x',z')}\sigma_{(x+x',z+z')\%2},$$
where the power $p$ of $\mathrm{i}$ in the prefactor is given by
$$p(x,z;x',z')=\sum_{i=1}^{N}\left(z_ix'_i-x_iz'_i + 2(z_i+z'_i)\left\lfloor\frac{x_i+x'_i}{2}\right\rfloor+2(x_i+x'_i)\left\lfloor\frac{z_i+z'_i}{2}\right\rfloor\right)\mod 4.$$
* **Commutation relation**: two Pauli operator either commute to anticommute.
$$\sigma_{(x,z)}\sigma_{(x',z')}=(-)^{c(x,z;x',z')}\sigma_{(x',z')}\sigma_{(x,z)},$$
where the *anticommutation indicator* $c$ has a simpler form
$$c(x,z;x',z')=\frac{p(x,z;x',z')-p(x',z';x,z)}{2}=\sum_{i=1}^{N}\left(z_ix'_i-x_iz'_i\right)\mod 2.$$

The binary vectors $x$ and $z$ can be interweaved into a $2N$-component vector $g=(x_0,z_0,x_1,z_1,\cdots)$, which forms the binary representation of a Pauli operator $\sigma_g$.

#### `Pauli` Class

`Pauli(g,p)` represents a Pauli operator.

**Parameters:**
* `g` binary representation of Pauli string.
* `p` phase indicator ($p=0,1,2,3$ stands for $i^p$ phase factor).

In [50]:
qst.pauli('iX').__dict__

{'g': array([1, 0]), 'p': 1}

#### `PauliList` Class

`PauliList(gs,ps)` represents a list of Pauli operators.

**Parameters:**
* `gs` array of binary representations of Pauli strings.
* `ps` array of phase indicators ($p=0,1,2,3$ stands for $i^p$ phase factor).

In [51]:
qst.paulis('XX','YY','ZZ').__dict__

{'gs': array([[1, 0, 1, 0],
        [1, 1, 1, 1],
        [0, 1, 0, 1]]),
 'ps': array([0, 0, 0])}

#### `PauliMonomial` Class

`PauliMonomial(g,p)` represents a Pauli operator with coefficient.

**Parameters:**
* `g` binary representation of Pauli string.
* `p` phase indicator ($p=0,1,2,3$ stands for $i^p$ phase factor).
* `c` coefficient (complex).

In [52]:
qst.pauli('iX').as_monomial().__dict__

{'g': array([1, 0]), 'p': 1, 'c': (1+0j)}

The property `c` can be set by the method `.set_c(c)`

In [53]:
qst.pauli('X').as_monomial().set_c(2.+0.j).__dict__

{'g': array([1, 0]), 'p': 0, 'c': (2+0j)}

#### `PauliPolynomial` Class

`PauliPolynomial(gs,ps)` represents a polynomial (linear combination) of Pauli operators.

**Parameters:**
* `gs` array of binary representations of Pauli strings.
* `ps` array of phase indicators ($p=0,1,2,3$ stands for $i^p$ phase factor).
* `cs` array of coefficients (complex).

In [54]:
(qst.pauli('XX') - 2*qst.pauli('YY')).__dict__

{'gs': array([[1, 0, 1, 0],
        [1, 1, 1, 1]]),
 'ps': array([0, 0]),
 'cs': array([ 1.+0.j, -2.+0.j])}

The property `cs` can be set by the method `.set_cs(cs)`